In [21]:
import torch 
import numpy as np 
import importlib 

In [ ]:
d_model = 128
x = torch.zeros(32, 64, 128)
n = 10000

seq_len = x.shape[1]

pe = torch.zeros(seq_len, d_model)

for k in range(0, seq_len):
    for i in range(0, d_model//2):
        theta = k/(n**(2*i/d_model))
        pe[k, 2*i] = np.sin(theta)
        pe[k, 2*i + 1] = np.cos(theta)

# saves to state_dict but doesn't do anything with optimizer 

print("PE shape: ", pe.shape)
# TODO: actually start using the positional encodings

In [ ]:
out = x + pe 
for i in range(0, out.shape[0]):
    assert (torch.allclose(out[i], pe))

In [49]:
batch_size = 32
d_model = 128 
seq_len = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch = torch.zeros([batch_size, seq_len]).to(int).to(device)

import models.byo_gpt as byogpt 
importlib.reload(byogpt )
model = byogpt.BYOGPT(50257).to(device)
out = model(batch)
print(out.shape)


Initializing BYOGPT with vocab size: 50257
Embed shape:  torch.Size([32, 64, 128])
PE shape:  torch.Size([64, 128])
After pos_embed shape:  torch.Size([32, 64, 128])
Attention shape:  torch.Size([32, 64, 128])
torch.Size([32, 64, 50257])
